# Exercise 1: Convolutional Neural Networks

## Import libraries

In [34]:
#%load_ext tensorboard

In [35]:
import zipfile
import os
import shutil
import random
import numpy as np
import signal
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
from tensorflow import keras
from keras.callbacks import TensorBoard
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.optimizers import Adam
#from google.colab import files
from IPython.display import Image

## Load files

In [36]:
# Path corresponding to the compressed dataset, must be pointing the zip file
path_zip = './state-farm-distracted-driver-detection.zip'

# Path corresponding to the folder generated in the colab environment after unzipping the dataset
path_extracted_files = './'

In [37]:
# Unzip the zip file state-farm-distracted-driver-detection.zip

# If the zip has been already uncompressed, it doesn't do it again
if os.path.exists(path_extracted_files):
    print("The zip file has already been extracted")
else:
    with zipfile.ZipFile(path_zip, 'r') as zip_ref:
        zip_ref.extractall("/content/extracted_files")

The zip file has already been extracted


## Configure training and test directories

In [38]:
# Delete the file driver_imgs_list.csv and sample_submission.csv
if os.path.exists(path_extracted_files + 'driver_imgs_list.csv'):
    os.remove(path_extracted_files + 'driver_imgs_list.csv')

if os.path.exists(path_extracted_files + 'sample_submission.csv'):
    os.remove(path_extracted_files + 'sample_submission.csv')

In [39]:
# Delete the folder imgs/test and its images
if os.path.exists(path_extracted_files + 'imgs/test'):
    shutil.rmtree(path_extracted_files + 'imgs/test')

In [40]:
# Make 2 directories named test and train in the extracted_files directory with the same folders that are in the train directory
if os.path.exists(path_extracted_files + 'test'):
    print("The directories have already been created")
else:
    os.makedirs(path_extracted_files + 'test')
    os.makedirs(path_extracted_files + 'train')

    # Get folders inside 'train'
    folders = os.listdir(path_extracted_files + 'imgs/train')

    for folder in folders:
        os.makedirs(path_extracted_files + f'test/{folder}')
        os.makedirs(path_extracted_files + f'train/{folder}')

The directories have already been created


## Generate train, validation and test sets using directory based labeling

In [41]:
# From the initial train directory, generate a new train and test directories, comprising 85% and 15% of the images respectively
folders = os.listdir(path_extracted_files + 'imgs/train')

# Check if the content/imgs/train/c0 folder is empty
if len(os.listdir(path_extracted_files + 'imgs/train/c0')) != 0:

    # Iterate through each folder (c0-c9) in the train directory
    for folder in os.listdir(path_extracted_files + 'imgs/train'):

        # List all images in the current folder
        images = os.listdir(path_extracted_files + f'imgs/train/{folder}')

        # Shuffle the list of images randomly
        random.shuffle(images)

        # Calculate the number of images to move to the test directory (15% of total)
        n = int(len(images) * 0.15)

        # Select the first n images as test images
        test_images = images[:n]

        # Move the selected test images to the test directory
        for img in test_images:
            shutil.move(path_extracted_files + f'imgs/train/{folder}/{img}', path_extracted_files + f'test/{folder}/{img}')

    # Move the remaining 85% of the images to the train directory
    for folder in os.listdir(path_extracted_files + 'imgs/train'):

        # List all images in the current folder
        images = os.listdir(path_extracted_files + f'imgs/train/{folder}')

        # Shuffle the list of images randomly
        random.shuffle(images)

        # Move all images to the train directory
        for img in images:
            shutil.move(path_extracted_files + f'imgs/train/{folder}/{img}', path_extracted_files + f'train/{folder}/{img}')
else:
    print("The images have already been moved")

The images have already been moved


## Configure the train and test sets

In [42]:
train_datagen = ImageDataGenerator(
    # We scale pixels between 0 and 1
    rescale=1./255,
    validation_split=0.1)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    # We define the directory where the training images are located
    path_extracted_files + 'train',
    # We transform all images to 96 x 96
    target_size=(96, 96),
    # We group the images into batches of 128
    batch_size=128,
    # We specify that the images remain in color (RGB)
    color_mode='rgb',
    # We define a categorical class type
    class_mode='categorical')

test_datagen = ImageDataGenerator(
    rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    # We define the directory where the test images are located
    path_extracted_files + 'test',
    # We transform all images to 96 x 96
    target_size=(96, 96),
    # We specify that the images remain in color (RGB)
    color_mode='rgb',
    # We define a categorical class type
    class_mode='categorical')

Found 19066 images belonging to 10 classes.
Found 3358 images belonging to 10 classes.


## Define the neural network architecture

In [43]:
# Definition of the image sizes
input_size = 96
num_colors = 3

# Definition of the network layers
layers = [
    # First convolution with 16 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(16,
                        (9,9),
                        activation='relu',
                        input_shape=(input_size,
                                    input_size,
                                    num_colors)),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2, 2),
    # Second convolution with 32 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(32,
                        (3,3),
                        activation='relu'),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2,2),
    # Third convolution with 64 filters of size 3 x 3 and relu activation function
    keras.layers.Conv2D(64,
                        (2,2),
                        activation='relu'),
    # Pooling of size 2 x 2 and stride 1
    keras.layers.MaxPooling2D(2,2),
    # Flattening to transform the information into a vector
    keras.layers.Flatten(),
    # Dense layer with 512 neurons and relu activation function
    keras.layers.Dense(512,
                       activation='relu'),
    # Dense layer with 512 neurons and relu activation function
    keras.layers.Dense(512,
                       activation='relu'),
    # Output layer with softmax activation function
    keras.layers.Dense(10,
                       activation = tf.nn.softmax),
]

model = keras.Sequential(
    layers,
    name="multi-class_classification")

c:\Users\usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


## Compile the model

In [44]:
# Optimization algorithm and loss function configuration
learning_rate = 0.001

# Create an instance of the Adam optimizer with your desired learning rate
optimizer = Adam(learning_rate=learning_rate)

# Compile your model with the optimizer
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Network structure
model.summary()


Model: "multi-class_classification"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 88, 88, 16)     │         3,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 44, 44, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 42, 42, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 21, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 20, 20, 64)     │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │     3,277,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,561,898 (13.59 MB)

 Trainable params: 3,561,898 (13.59 MB)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [45]:

# Training process execution
# Iterations: 15
# Number of steps per iteration: 4
history = model.fit(train_generator,
                    steps_per_epoch = 4,
                    epochs = 50)

Epoch 1/50


c:\Users\usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 333ms/step - accuracy: 0.1273 - loss: 2.3374
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 332ms/step - accuracy: 0.1161 - loss: 2.2905
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 339ms/step - accuracy: 0.1276 - loss: 2.2750
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - accuracy: 0.1234 - loss: 2.2652
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 338ms/step - accuracy: 0.2198 - loss: 2.2235
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - accuracy: 0.2383 - loss: 2.1109
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 347ms/step - accuracy: 0.2331 - loss: 2.0385
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 327ms/step - accuracy: 0.2422 - loss: 2.0064
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 346ms/step - accuracy: 0.3615 - loss: 1.7825
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 372ms/step - accuracy: 0.4164 - loss: 1.6307
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 368ms/step - accuracy: 0.4586 - loss: 1.5614
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 338ms/step - accuracy: 0.5451 - loss: 1.3577


c:\Users\usuario\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 378ms/step - accuracy: 0.9354 - loss: 0.2088
Epoch 40/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 410ms/step - accuracy: 0.9516 - loss: 0.2273
Epoch 41/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 381ms/step - accuracy: 0.9211 - loss: 0.2150
Epoch 42/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 360ms/step - accuracy: 0.9544 - loss: 0.2171
Epoch 43/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 354ms/step - accuracy: 0.9513 - loss: 0.1889
Epoch 44/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 335ms/step - accuracy: 0.9328 - loss: 0.2211
Epoch 45/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.9622 - loss: 0.1926
Epoch 46/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 339ms/step - accuracy: 0.9474 - loss: 0.1683
Epoch 47/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - accuracy: 0.9638 - loss: 0.1181
Epoch 48/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 340ms/step - accuracy: 0.9734 - loss: 0.1073
Epoch 49/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 336ms/step - accuracy: 0.9661 - loss: 0.1293
Epoch 50/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 334ms/step - accuracy: 0.9563 - loss:

## Test the model

In [49]:
# Evaluation of the model using the test set
results = model.evaluate(test_generator)

print("Test Loss:", results[0])
print("Test Accuracy:", results[1])
print("Misclassification:", 1 - results[1])

predictions = []
true_labels = []

for i in range(len(test_generator)):
    # Get batch of data and labels and predict on it
    batch_data, batch_labels = test_generator[i]
    batch_predictions = model.predict(batch_data, verbose=0)
    # Append predictions and true labels
    predictions.extend(np.argmax(batch_predictions, axis=1))
    true_labels.extend(np.argmax(batch_labels, axis=1))

cm = confusion_matrix(true_labels, predictions)


# Convert to numpy array
cm = np.array(cm)

# Calculate total samples
total_samples = np.sum(cm)

# Calculate TP, TN, FP, FN
TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP
TN = total_samples - (TP + FP + FN)

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

# Calculate average precision and recall
average_precision = np.mean(precision)
average_recall = np.mean(recall)

print(f'Average Precision: {average_precision}')
print(f'Average Recall: {average_recall}')



105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.9685 - loss: 0.1226
Test Loss: 0.14389590919017792
Test Accuracy: 0.9621798396110535
Misclassification: 0.03782016038894653
Average Precision: 0.9635470588724525
Average Recall: 0.9621814490623487


## Create the confusion matrix

In [48]:
predictions = []
true_labels = []

# Iterate over each batch in the test generator
for i in range(len(test_generator)):
    # Get batch of data and labels and predict on it
    batch_data, batch_labels = test_generator[i]
    batch_predictions = model.predict(batch_data, verbose=0)
    # Append predictions and true labels
    predictions.extend(np.argmax(batch_predictions, axis=1))
    true_labels.extend(np.argmax(batch_labels, axis=1))

cm = confusion_matrix(true_labels, predictions)


labels = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
disp = ConfusionMatrixDisplay(cm, display_labels=labels)
disp.plot()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Convert to numpy array
cm = np.array(cm)

# Calculate total samples
total_samples = np.sum(cm)

# Calculate TP, TN, FP, FN
TP = np.diag(cm)
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP
TN = total_samples - (TP + FP + FN)

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

# Calculate accuracy and error rate
accuracy = (TP + TN) / total_samples
error_rate = 1 - accuracy
# Calculate average precision and recall
average_precision = np.mean(precision)
average_recall = np.mean(recall)

print(f'Average Precision: {average_precision}')
print(f'Average Recall: {average_recall}')

KeyboardInterrupt: 

## Inference process

In [ ]:
label_names = ['safe driving', 'texting - right', 'talking on the phone - right', 'texting - left', 'talking on the phone - left', 'operating the radio', 'drinking', 'reaching behind', 'hair and makeup', 'talking to passenger']
minimum_value = 0.78

# We use the file insertion system of Colab
uploaded = files.upload()

for filename, filedata in uploaded.items():
    # Display the image using IPython.display.Image
    display(Image(data=filedata, width=300))

    # Image path configuration
    path = '/content/' + filename

    # Preprocess the image
    img = image.load_img(path, target_size=(96, 96))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Image transformation into a tensor
    image_tensor = np.vstack([x])

    # Scale the data between 0 and 1
    image_tensor /= 255

    # Inference execution
    classes = model.predict(image_tensor)

    # Get the indices of top three classes
    top_three_indices = np.argsort(classes[0])[::-1][:3]

    # Print the top three classes and their probabilities
    print(f"{filename} is classified as:")
    for i in top_three_indices:
        print(f"   - {label_names[i]} with probability {classes[0][i]}")

    # If no class meets the threshold
    if not any(classes[0][i] > minimum_value for i in range(len(classes[0]))):
        print(filename + ' is not classified in any class.')


NameError: name 'files' is not defined